In [1]:
from importlib import reload
import mapreduce
reload(mapreduce)
from mapreduce import FilteringMapReduce, map_reduce_df
import pandas as pd
import utils
reload(utils)
from utils import get_date_index, get_days_in_range

In [2]:
# fill with median from previous two weeks
days = get_days_in_range('2017-06-16', '2017-08-15')

In [3]:
cols = ['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion']
types = {'id': 'int32',
         'date': 'int16',
         'item_nbr': 'int32',
         'store_nbr': 'int16',
         'unit_sales': 'float32',
         'onpromotion': bool}
mapreduce = FilteringMapReduce(lambda df: df[df.date.isin(days)], cols, types)
train = map_reduce_df('./data/train_processed.csv', mapreduce, types=types, position=110000000, cols=cols, verbose=True)

Reading from position 110000000, batch size 10000000...
Filtering 10000000...
Filtered 930790, mapping...
Mapped, reducing...
Done.
Reading from position 120000000, batch size 10000000...
Filtering 5497041...
Filtered 5497041, mapping...
Mapped, reducing...
Done.
End of dataset is found.


In [4]:
train.ix[train.unit_sales < 0, 'unit_sales'] = 0
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(["date", "store_nbr", "item_nbr"], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=["date", "store_nbr", "item_nbr"]
    )
)
train.loc[:, "unit_sales"].fillna(0, inplace=True)
# Assume missing entris imply no promotion
train.loc[:, "onpromotion"].fillna("False", inplace=True)
train.head()

id  unit_sales onpromotion
date   store_nbr item_nbr                                      
1635.0 46.0      1695836.0  119999999.0   14.421875       False
                 1695837.0  120000000.0   11.109375       False
                 1695840.0  120000001.0    4.000000       False
                 1695845.0  120000002.0   44.000000       False
                 1695846.0  120000003.0    3.345703       False

In [6]:
train.reset_index(inplace=True)

In [7]:
days_train1 = get_days_in_range('2017-06-16', '2017-06-30')
days_train2 = get_days_in_range('2017-07-01', '2017-07-15')
days_train3 = get_days_in_range('2017-07-16', '2017-07-31')
days_test = get_days_in_range('2017-08-01', '2017-08-15')

medians_train1 = train[train.date.isin(days_train1)]
medians_train2 = train[train.date.isin(days_train2)]
medians_train3 = train[train.date.isin(days_train3)]
medians_test = train[train.date.isin(days_test)]
del train

In [8]:
import metric
reload(metric)
from metric import NWRMSLE, get_weights
import numpy as np

def ev(train, test):
    median = train.groupby(by=['item_nbr','store_nbr'], as_index=False).agg({'unit_sales':'median'})
    median['unit_sales_median'] = median['unit_sales']
    del median['unit_sales']
    test = test.merge(median, on=['item_nbr','store_nbr'], how='left')
    test.fillna(0.0, inplace=True)
    y_true = test['unit_sales']
    y_pred = test['unit_sales_median']
    weights = get_weights(test['item_nbr'])
    error = NWRMSLE(y_true, y_pred, weights)
    return error

errors = []
errors.append(ev(medians_train1, medians_train2))
errors.append(ev(medians_train2, medians_train3))
errors.append(ev(medians_train3, medians_test))
np.mean(errors)

0.62636099248854926

In [9]:
errors

[0.6302102004724058, 0.626762948794633, 0.6221098281986088]

In [10]:
test = pd.read_csv('./data/test.csv')
median = medians_test.groupby(by=['item_nbr','store_nbr'], as_index=False).agg({'unit_sales':'median'})
median['unit_sales_median'] = median['unit_sales']
del median['unit_sales']
test = test.merge(median, on=['item_nbr','store_nbr'], how='left')
test.fillna(0.0, inplace=True)
test['unit_sales'] = test['unit_sales_median']
del test['unit_sales_median']

In [11]:
test.ix[test.unit_sales < 0, 'unit_sales'] = 0

In [12]:
# 0.59
test[['id', 'unit_sales']].to_csv('./submissions/previous_week_median_0.62.csv', index=False)